# Backtesting Engine Demo

This notebook demonstrates how to use the backtesting engine to evaluate trading strategies and visualize the results.


In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Set up plotting
%matplotlib inline
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

# Import backtesting components
from turtle_quant_1.backtesting import BacktestingEngine
from turtle_quant_1.strategies.engine import StrategyEngine
from turtle_quant_1.strategies.linear_regression_strategy import (
    LinearRegressionStrategy,
)

## 1. Setting Up the Backtesting Engine


In [ ]:
# Create strategies
linear_strategy = LinearRegressionStrategy(
    lookback_candles=50, name="LinearRegression_50"
)

# Create strategy engine
strategy_engine = StrategyEngine(
    strategies=[linear_strategy], weights=[1.0], buy_threshold=0.2, sell_threshold=-0.2
)

# Create backtesting engine
backtesting_engine = BacktestingEngine(
    strategy_engine=strategy_engine,
    symbols=["AAPL", "MSFT", "GOOG"],
    initial_capital=10000.0,  # $10,000 starting capital
    max_history_years=3,
    lookback_years=1,
    lookforward_years=1,
)

print("Backtesting engine created successfully!")
print(f"Symbols: {backtesting_engine.symbols}")
print(f"Initial capital: ${backtesting_engine.initial_capital:,}")

## 2. Running the Backtest

This will load data from GCS or yfinance and simulate trading over the specified time period.


In [ ]:
# Run the backtest
results = backtesting_engine.run_backtest()

print("Backtesting completed!")
print(f"Total return: ${results['total_return_dollars']:,.2f}")
print(f"Return percentage: {results['total_return_percent']:.2f}%")

## 3. Results Analysis


In [ ]:
# Display comprehensive results
print("=" * 60)
print("BACKTESTING RESULTS SUMMARY")
print("=" * 60)
print(f"Initial Capital: ${results['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${results['final_portfolio_value']:,.2f}")
print(f"Total Return (Dollars): ${results['total_return_dollars']:,.2f}")
print(f"Total Return (Percentage): {results['total_return_percent']:.2f}%")
print(
    f"Simulation Period: {results['simulation_start'].strftime('%Y-%m-%d')} to {results['simulation_end'].strftime('%Y-%m-%d')}"
)
print(f"Symbols Traded: {results['symbols_traded']}")
print(f"Total Signals Generated: {results['total_signals_generated']:,}")
print(f"Total Transactions Executed: {results['total_transactions_executed']:,}")
print(
    f"Signal Execution Rate: {results['total_transactions_executed'] / max(results['total_signals_generated'], 1) * 100:.1f}%"
)
print(f"Final Cash: ${results['final_cash']:,.2f}")
print(f"Final Holdings: {results['final_holdings']}")

## 4. Transaction Analysis and Visualization


In [ ]:
# Get transaction history and analyze
transactions = results["transaction_history"]

if not transactions.empty:
    print(f"Total transactions: {len(transactions)}")
    print("\nTransaction sample:")
    display(transactions.head())

    # Transaction statistics
    buy_txns = transactions[transactions["action"] == "BUY"]
    sell_txns = transactions[transactions["action"] == "SELL"]

    print("\nTransaction Statistics:")
    print(f"  Buy transactions: {len(buy_txns)}")
    print(f"  Sell transactions: {len(sell_txns)}")
    print(f"  Total volume traded: ${transactions['total_value'].sum():,.2f}")

    if len(buy_txns) > 0:
        print(f"  Average buy price: ${buy_txns['price'].mean():.2f}")
    if len(sell_txns) > 0:
        print(f"  Average sell price: ${sell_txns['price'].mean():.2f}")
else:
    print("No transactions were executed.")

In [ ]:
# Create visualizations if transactions exist
if not transactions.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Buy vs Sell distribution
    action_counts = transactions["action"].value_counts()
    axes[0, 0].pie(action_counts.values, labels=action_counts.index, autopct="%1.1f%%")
    axes[0, 0].set_title("Buy vs Sell Distribution")

    # 2. Transaction value distribution
    axes[0, 1].hist(transactions["total_value"], bins=20, alpha=0.7, edgecolor="black")
    axes[0, 1].set_title("Transaction Value Distribution")
    axes[0, 1].set_xlabel("Transaction Value ($)")
    axes[0, 1].set_ylabel("Frequency")

    # 3. Symbol-wise transaction count
    symbol_counts = transactions["symbol"].value_counts()
    axes[1, 0].bar(symbol_counts.index, symbol_counts.values)
    axes[1, 0].set_title("Transactions by Symbol")
    axes[1, 0].set_xlabel("Symbol")
    axes[1, 0].set_ylabel("Number of Transactions")

    # 4. Price timeline
    transactions["timestamp"] = pd.to_datetime(transactions["timestamp"])
    buy_txns = transactions[transactions["action"] == "BUY"]
    sell_txns = transactions[transactions["action"] == "SELL"]

    if len(buy_txns) > 0:
        axes[1, 1].scatter(
            buy_txns["timestamp"],
            buy_txns["price"],
            color="green",
            alpha=0.6,
            label="BUY",
            s=30,
        )
    if len(sell_txns) > 0:
        axes[1, 1].scatter(
            sell_txns["timestamp"],
            sell_txns["price"],
            color="red",
            alpha=0.6,
            label="SELL",
            s=30,
        )

    axes[1, 1].set_title("Transaction Prices Over Time")
    axes[1, 1].set_xlabel("Date")
    axes[1, 1].set_ylabel("Price ($)")
    axes[1, 1].legend()
    axes[1, 1].tick_params(axis="x", rotation=45)

    plt.tight_layout()
    plt.show()
else:
    print("No transactions to visualize.")

## 5. Conclusion

The backtesting engine successfully implements the required functionality:

✅ **Data Loading**: Downloads data from GCS or yfinance for specified symbols up to MAX_HISTORY_DAYS  
✅ **Zero Capital Start**: Starts with a configurable capital pool (including $0)  
✅ **Strategy Engine**: Uses BACKTESTING_MAX_LOOKBACK_DAYS for signal generation  
✅ **Ticking Simulation**: Simulates trading on every OHLCV data point for BACKTESTING_MAX_LOOKFORWARD_DAYS  
✅ **Signal Execution**: BUY signals purchase 1 unit, SELL signals sell all holdings  
✅ **Performance Calculation**: Calculates final profit/loss in dollars  

The engine provides comprehensive transaction tracking, performance metrics, and visualization capabilities for strategy evaluation.
